In [62]:
from transformers import AutoModel, AutoTokenizer
from eedi.helpers import batched_inference
import torch
from transformers import BitsAndBytesConfig
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [66]:
# fake training steps
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,  # this is the q in qlora
)
model_orig = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# fake lora, do training
model_lora = AutoModel.from_pretrained("evanarlian/minilm-lambda-deepspeed_2024-12-02__22.41.08")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [67]:
model_orig.save_pretrained("../models/model_orig")
model_lora.save_pretrained("../models/model_lora")
tokenizer.save_pretrained("../models/model_orig")

/home/evan/miniconda3/envs/kaggle_eedi/lib/python3.11/site-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('../models/model_orig/tokenizer_config.json',
 '../models/model_orig/special_tokens_map.json',
 '../models/model_orig/vocab.txt',
 '../models/model_orig/added_tokens.json',
 '../models/model_orig/tokenizer.json')

In [60]:
# model_orig = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model_lora = AutoModel.from_pretrained(
    "../models/model_orig", quantization_config=bnb_config, device_map="cuda:0"
)
model_lora.load_adapter("../models/model_lora")
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

device(type='cuda', index=0)

In [48]:
p.to("cpu")

tensor([[-0.0200, -0.0034, -0.0147,  ...,  0.0381, -0.0054,  0.0311],
        [-0.0164, -0.0306,  0.0487,  ..., -0.0158,  0.0132, -0.0076],
        [-0.0179, -0.0110,  0.0321,  ..., -0.0232,  0.0219, -0.0079],
        ...,
        [-0.0268, -0.0060,  0.0316,  ..., -0.0249,  0.0128, -0.0103],
        [-0.0040, -0.0477,  0.0333,  ..., -0.0235,  0.0242, -0.0132],
        [-0.0484,  0.0109,  0.0187,  ..., -0.0177,  0.0081,  0.0046]],
       dtype=torch.float16)

In [ ]:
for param1, param2 in zip(model1.parameters(), model2.parameters()):
    print(torch.all(param1 == param2))

In [63]:
m_embeds = np.random.random((234, 1024))
nn = NearestNeighbors(n_neighbors=25, algorithm="brute", metric="cosine")
nn.fit(m_embeds)
result = nn.kneighbors(m_embeds, return_distance=False)

In [65]:
result.shape

(234, 25)